In [3]:
import tushare as ts
import numpy as np
ts.set_token("8c6e5026fd3e2b5f96f15553d72f84eaaea8182a9157b783a41b1be0")
pro = ts.pro_api()

In [73]:
all_codes = list(set(pro.cb_basic(fields="ts_code").ts_code))
all_codes

['113520.SH',
 '110232.SH',
 '110043.SH',
 '110079.SH',
 '100177.SH',
 '113018.SH',
 '100096.SH',
 '110813.SH',
 '123027.SZ',
 '123075.SZ',
 '128134.SZ',
 '125822.SZ',
 '124009.SZ',
 '128117.SZ',
 '110011.SH',
 '110050.SH',
 '123010.SZ',
 '113587.SH',
 '110072.SH',
 '113596.SH',
 '127037.SZ',
 '113040.SH',
 '128145.SZ',
 '128082.SZ',
 '113533.SH',
 '113043.SH',
 '128118.SZ',
 '110016.SH',
 '113536.SH',
 '113553.SH',
 '123083.SZ',
 '123101.SZ',
 '110047.SH',
 '128133.SZ',
 '128103.SZ',
 '128127.SZ',
 '123106.SZ',
 '110040.SH',
 '128041.SZ',
 '128141.SZ',
 '123107.SZ',
 '100117.SH',
 '110418.SH',
 '121002.SZ',
 '110061.SH',
 '110025.SH',
 '124012.SZ',
 '100795.SH',
 '128026.SZ',
 '110077.SH',
 '127034.SZ',
 '128079.SZ',
 '113573.SH',
 '126729.SZ',
 '110041.SH',
 '113027.SH',
 '128006.SZ',
 '128102.SZ',
 '128060.SZ',
 '113502.SH',
 '113556.SH',
 '128017.SZ',
 '128126.SZ',
 '127006.SZ',
 '123111.SZ',
 '124006.SZ',
 '124007.SZ',
 '128021.SZ',
 '110032.SH',
 '113581.SH',
 '113552.SH',
 '1100

### 记录每只股票转股价变化表

In [74]:
# import os
# from tqdm import tqdm
# # all_codes = get_all_codes()
# os.chdir(r'E:\quantTradeLeanner\QuantTrader\tushare\convert_table')
# os.getcwd()
# import time # 每分钟最多访问200次
# for code in tqdm(all_codes):
#     df = pro.cb_price_chg(ts_code=code,fields="change_date,convert_price_initial,convertprice_aft")
#     df.to_csv(code.split('.')[0]+'_'+code.split('.')[1]+'.csv')
#     time.sleep(0.1)
#     del df

100%|████████████████████████████████████████████████████████████████████████████████| 648/648 [04:32<00:00,  2.38it/s]


In [131]:
### 加载所有转债的代码
# def get_all_codes():
#     '''
#         return: 加载所有转债的代码
#     '''
#     os.chdir(r'E:\quantTradeLeanner\QuantTrader\tushare')
#     all_codes = np.load('all_codes_20180101_20210601.npy')
#     return all_codes

### 获取债股票代码转换字典
def get_d2s_dict(all_codes):
    '''
        all_codes: 这两年所有的债券代码
        return: 这两年所有的债券转股票代码
    '''
    df = pro.cb_basic(fields="ts_code,stk_code")
#     df = df[df.ts_code.isin(all_codes)]
    df.reset_index()
    det2share_dict = df.set_index('ts_code').T.to_dict('list')
    return det2share_dict

### 查询债券价格
def get_daily_table(time):
    '''
        time: 时间
        return : 两列df, 债券代码和债券价格
    '''
    daily_table = pro.cb_daily(trade_date=time,fields='ts_code,close')
    daily_table.columns = ['ts_code','det_price']
    return daily_table

# 查询股票价格
def get_share_price(codes,time,det2share_dict):
    '''
        code: 当日代码列表
        time: 检索时间
        det2share_dict: 债券对应的正股代码字典
        return : pd.DataFrame 当日的正股价格序列
        作用: 检索当日可转债对应的正股价
    '''
    share_price = []
    for code in codes:
        share_code = det2share_dict[code][0]
        tt = np.array(pro.daily(ts_code=share_code, start_date=time, end_date=time).close)
        if len(tt) == 0:
            share_price.append(0)
        else:
            share_price.append(tt[0])
            
#         share_price.append(np.array(pro.daily(ts_code=share_code, start_date=time, end_date=time).pre_close))
#         print(np.array(pro.daily(ts_code=share_code, start_date=time, end_date=time).pre_close))
#         print(pro.daily(ts_code=share_code, start_date=time, end_date=time).pre_close)
#     break
    share_price = pd.DataFrame(share_price,columns=['share_price'])
    return  share_price

# 查询转股价格
def get_convprice_per_det(codes,time):
    '''
        code: 当日代码列表
        time: 检索时间
        return : pd.DataFrame 当日的转股价序列
        作用: 检索当日可转债对应的转股价
    '''
    os.chdir(r'E:\quantTradeLeanner\QuantTrader\tushare\convert_table')
    conv_price_list = []
    for code in codes:
        code_info = pd.read_csv(code.split('.')[0]+'_'+code.split('.')[1]+'.csv')
        date_list = list(code_info.change_date)
        price_list = list(code_info.convertprice_aft)
        price_list[0]=code_info.convert_price_initial[0]
        flag = 0
        i = 0
        for d in date_list:
            i = i + 1
            if int(time) > int(d):
                flag = i
            else:
                break
        #     if flag == 0 or flag ==1:
        conv_price = price_list[flag-1]
        conv_price_list.append(conv_price)
    conv_price_list = pd.DataFrame(conv_price_list,columns=['conv_price'])
    return conv_price_list
def get_final_table(daily_table,share_price,conv_price):
    '''
        return: 最终表格,包括溢价率
    '''
    df = pd.concat([daily_table,share_price,conv_price],axis=1)
    mid_value = (100 / df.conv_price) * df.share_price
    over_rate = (df.det_price-mid_value)/mid_value
    over_rate = pd.DataFrame(over_rate,columns=['over_rate'])
    df1 = pd.concat([df,over_rate],axis=1)
    df1 = df1.sort_values(by='over_rate',ascending=False).reset_index()
    return df1

def save_table(start_time,end_time):
    
    for time in tqdm(range(start_time,end_time)):
     
        is_empty = True if len(pro.cb_daily(trade_date=str(time),fields='ts_code')) == 0 else False 
        if is_empty:
            pass
        else:
            daily_table = get_daily_table(time)
            share_price = get_share_price(daily_table.ts_code,time,det2share_dict)
            conv_price = get_convprice_per_det(daily_table.ts_code,time)
            df = get_final_table(daily_table,share_price,conv_price)
            os.chdir(r'E:\quantTradeLeanner\QuantTrader\tushare\daily_table')
            df.to_csv(str(time)+'.csv')
            print(str(time)+'.csv saved!')

In [140]:
# start_time,end_time = 20181228,20181231
# save_table(start_time,end_time)

In [141]:
# start_time,end_time = 20190329,20191231
# save_table(start_time,end_time)

In [142]:
# start_time,end_time = 20200101,20201231
# save_table(start_time,end_time)

In [143]:
# start_time,end_time = 20210101,20210613
# save_table(start_time,end_time)

### 获取20180404当天的溢价率表

In [144]:
import numpy as np 
import os 
from tqdm import tqdm
import pandas as pd


all_codes = get_all_codes()
det2share_dict = get_d2s_dict(all_codes)

time = '20180404'
daily_table = get_daily_table(time)
share_price = get_share_price(daily_table.ts_code,time,det2share_dict)
conv_price = get_convprice_per_det(daily_table.ts_code,time)
get_final_table(daily_table,share_price,conv_price)

,index,ts_code,det_price,share_price,conv_price,over_rate
0,26,113502.SH,98.710,0.00,45.48,inf
1,24,113016.SH,102.100,0.00,23.00,inf
2,52,123007.SZ,117.050,0.00,45.21,inf
3,12,128013.SZ,92.385,4.58,10.00,1.017140
4,9,113010.SH,105.300,4.86,9.30,1.015000
...,...,...,...,...,...,...
57,49,110042.SH,114.690,15.76,14.29,0.039924
58,61,128036.SZ,100.032,9.35,9.72,0.039905
59,56,123008.SZ,135.200,65.28,47.01,-0.026386
60,28,110039.SH,139.440,26.61,18.36,-0.037911


### 数据填充和数据清洗

In [146]:
os.chdir(r'E:\quantTradeLeanner\QuantTrader\tushare\daily_table')
os.getcwd()

'E:\\quantTradeLeanner\\QuantTrader\\tushare\\daily_table'

In [150]:
file_list = os.listdir()
len(file_list)

834

In [177]:


lost_info = []
for file in tqdm(file_list):
#     file = '20180404.csv'
    df = pd.read_csv(file)
    df = df.replace([np.inf, -np.inf], np.nan)
    if df.over_rate.isna().sum()>0:
        lost_dict = {
            'file':file,
            'nan_nums':df.over_rate.isna().sum(),
            'nan_det_codes':list(df[df.over_rate.isna()].ts_code)
        }
        lost_info.append(lost_dict)
    else:
        pass

# break

100%|████████████████████████████████████████████████████████████████████████████████| 834/834 [00:11<00:00, 70.76it/s]


In [173]:
list(df[df.over_rate.isna()].ts_code)

['113502.SH', '113016.SH', '123007.SZ']

In [190]:
all_nums = 0
all_lost_codes = set()
for info in tqdm(lost_info):
#     print(info)
    all_nums += info['nan_nums']
    all_lost_codes.update(info['nan_det_codes'])
print(all_nums)

100%|████████████████████████████████████████████████████████████████████████████| 400/400 [00:00<00:00, 400124.40it/s]

572


In [194]:
lost_info

[{'file': '20180108.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180109.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180110.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180111.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180112.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180115.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180116.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180117.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180118.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180119.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180122.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180123.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180124.csv', 'nan_nums': 1, 'nan_det_codes': ['113502.SH']},
 {'file': '20180125.csv', 'nan_nums': 

In [192]:
len(all_lost_codes)

44

In [193]:
all_lost_codes

{'110033.SH',
 '110048.SH',
 '110053.SH',
 '110056.SH',
 '110067.SH',
 '110071.SH',
 '110077.SH',
 '113008.SH',
 '113015.SH',
 '113016.SH',
 '113027.SH',
 '113502.SH',
 '113515.SH',
 '113517.SH',
 '113524.SH',
 '113527.SH',
 '113537.SH',
 '113552.SH',
 '113576.SH',
 '113595.SH',
 '123004.SZ',
 '123007.SZ',
 '123010.SZ',
 '123013.SZ',
 '123021.SZ',
 '123031.SZ',
 '123043.SZ',
 '123044.SZ',
 '123052.SZ',
 '123061.SZ',
 '123068.SZ',
 '127018.SZ',
 '127025.SZ',
 '127030.SZ',
 '128010.SZ',
 '128012.SZ',
 '128022.SZ',
 '128036.SZ',
 '128037.SZ',
 '128040.SZ',
 '128043.SZ',
 '128054.SZ',
 '128086.SZ',
 '128090.SZ'}